# Interactive Dashboard with HoloViz

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 20/03/2025   | Martin | Create  | Created notebook | 
| 21/03/2025   | Martin | Create  | Completed data cleaning process | 
| 22/03/2025   | Martin | Create  | Completed data cleaning process | 

# Content

* [Introduction](#introduction)
* [Setup](#setup)
* [The Data](#the-data)
* [The Dashboard](#the-dashboard)

# Introduction

This notebook complements the associated article to create interactive dashboards using components from the HoloViz Suite, more specifically, Panel, hvPlot and HoloViews. These are data visualisations that allow for user interaction to dive deeper into the data.

In this tutorial, we will perform some data cleaning prior to the visualisation tutorial and explore the Graduate Employment Survey (GES) conducted by Universities in Singapore from 2013-2022.

# Setup

If you have `poetry` installed you can install install dependencies directly after cloning the project:

```
poetry install
```

Otherwise you can use the `requirements.txt` inside the repo using `pip` after creating a virtual environment

```
pip install -r requirements.txt
```

# The Data

_More details about the data can be found in the article or from the [source](https://data.gov.sg/datasets/d_3c55210de27fcccda2ed0c63fdd2b352/view)_

The Graduate Employment Survey (GES) is jointly conducted by NTU, NUS, SMU, SIT, SUTD and SUSS (Universities in Singapore) annually to survey the employment conditions of graduates about six months after their final examinations. The results are published by the Ministry of Education (MOE)

Here we will perform some data cleaning to ensure a good format for visualisation later.

In [ ]:
import pandas as pd
import numpy as np
import re
import hvplot.pandas
import panel as pn
import holoviews as hv
from datetime import datetime

pn.extension()

In [250]:
df = pd.read_csv("data/GraduateEmploymentSurveyNTUNUSSITSMUSUSSSUTD.csv")
df.head()

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile
0,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,97.4,96.1,3701,3200,3727,3350,2900,4000
1,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),97.1,95.7,2850,2700,2938,2700,2700,2900
2,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),90.9,85.7,3053,3000,3214,3000,2700,3500
3,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business and Computing,87.5,87.5,3557,3400,3615,3400,3000,4100
4,2013,Nanyang Technological University,College of Engineering,Aerospace Engineering,95.3,95.3,3494,3500,3536,3500,3100,3816


First we'll perform some basic data sanitisation - removing NA values and declaring the column data types

In [251]:
# Remove the rows with na (str)
df = df[df['employment_rate_overall'] != 'na']

# Specify the datatypes of columns
cols_to_change = [col for col in df.columns if col not in 
                  ['year', 'university', 'school', 'degree']]
df = df.astype(dtype={
  col: 'float64' for col in cols_to_change
})

Next, we observe that for each University there might be different names for the degrees and schools, and for degrees there are distinctions between the level of degree obtained (Hons, Cum Laude, etc.). As such, we attempt to remove these distinctions and separate them into different columns to ensure better separation

_Note: The cleaning might not be 100% accurate, so please excuse some missed distinctions_

In [252]:
# Change the datatype of year column
df['year'] = pd.to_datetime(df['year'], format="%Y")

In [253]:
# Cleaning the school column
# Remove any details from brackets
df['school'] = df['school'].str.replace(r'\(.*?\)', '', regex=True)

# Remove any special characters from the back
df['school'] = df['school'].str.replace(r'[\*|\\|\#]+', '', regex=True)

# Remove the white space after dash
df['school'] = df['school'].str.replace(r'-\s', '-', regex=True)

# Remove leading and trailing whitespace
df['school'] = df['school'].str.strip()

In [254]:
# Cleaning the degree column
# Remove special characters from the back
df['degree'] = df['degree'].str.replace(r'[\*|\\|\#|\^|\.]+', '', regex=True)

# Extract out if they were honours or cum laude programs
df['advanced'] = np.where(df['degree'].str.contains(r'Honours|\(Hons\)|Cum\s+Laude'), 1, 0)
remove_advanced = r'\s+with\s+Honours|\(Hons\)|\(?Cum\sLaude\sand\sabove\)?'
df['degree'] = df['degree'].str.replace(remove_advanced, '', regex=True)

# Remove the length of degree
df['degree'] = df['degree'].str.replace(r'\([^()]*\d[^()]*\)', '', regex=True)

# Remove non-degree related terms
df['degree'] = df['degree'].str.replace(r'\(LLB\)|\(MBBS\)|\(Land\)', '', regex=True)

# Some degree types are hidden between brackets so we extract them
temp = df['degree'].str.extract(r'\(([^)]+)\)')
df.loc[temp[~temp[0].isna()].index, 'degree'] = temp[~temp[0].isna()][0]

# Some degrees are also only expressed after the word "in"
temp = df['degree'].str.extract(r'\bin\b\s+(.*?)$')
df.loc[temp[~temp[0].isna()].index, 'degree'] = temp[~temp[0].isna()][0]

# Remove term "Bachelor of"
df['degree'] = df['degree'].str.replace(r'Bachelor\sof\s?', '', regex=True, case=False)

# Replace some special characters with their word equivalents
df['degree'] = df['degree'].str.replace('&', 'and')
df['degree'] = df['degree'].str.replace('/', ' and ')
df['degree'] = df['degree'].str.replace('with', '')
df['degree'] = df['degree'].str.replace(r'\s+', ' ', regex=True)
df['degree'] = df['degree'].str.replace(r's$', '', regex=True)

# Remove leading and trailing whitespace
df['degree'] = df['degree'].str.strip()

# Reset the index
df = df.reset_index(drop=True)

In [255]:
df.head()

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile,advanced
0,2013-01-01,Nanyang Technological University,College of Business,Accountancy and Busines,97.4,96.1,3701.0,3200.0,3727.0,3350.0,2900.0,4000.0,0
1,2013-01-01,Nanyang Technological University,College of Business,Accountancy,97.1,95.7,2850.0,2700.0,2938.0,2700.0,2700.0,2900.0,1
2,2013-01-01,Nanyang Technological University,College of Business,Business,90.9,85.7,3053.0,3000.0,3214.0,3000.0,2700.0,3500.0,1
3,2013-01-01,Nanyang Technological University,College of Business,Business and Computing,87.5,87.5,3557.0,3400.0,3615.0,3400.0,3000.0,4100.0,0
4,2013-01-01,Nanyang Technological University,College of Engineering,Aerospace Engineering,95.3,95.3,3494.0,3500.0,3536.0,3500.0,3100.0,3816.0,0


---

# The Dashboard

Now that the data is prepared, we'll create individual plots while describing how each of them can be used to interpret some details about the data set. Then we'll combine into a complete dashboard for viewing.

In [24]:
# You can load the data using this function if you want to jump straight to the dashboard

## Create the control scheme

We create controls across the different columns that will allow the users to filter the data according what they want to observe

In [146]:
# Create various selection groups
universities = list(df['university'].unique())
years = list(df['year'].unique())
degrees = list(df['degree'].unique())
advanced = list(df['advanced'].unique())

COLOURS = ['#264653', '#e9c46a', '#e76f51', '#2a9d8f', '#f4a261']

In [ ]:
# Checkbox group for universities
## Allows users to select multiple universities at the same time to compare them
uni_select = pn.widgets.Select(
  name="Select Universities:",
  value=universities,
  options=universities,
)

## Allow users to select the range of years they want to see
year_slider = pn.widgets.DateRangeSlider(
  name='Graduate Date Range',
  start=years[0],
  end=years[-1],
  value=(years[0], years[-1]),
  step=1,
  format="%Y"
)

## Allow users to select multiple degrees at the same time
degree_multi_select = pn.widgets.MultiChoice(
  name='Degrees to Compare (limit 5)',
  options=degrees,
  max_items=5
)

pn.Column(
  uni_select,
  pn.Row(year_slider, degree_multi_select, switch)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b85fff2f-24e3-4036-8d0e-cc4376bdc918': {'version…

In [55]:
year_slider.value

(Timestamp('2013-01-01 00:00:00'), Timestamp('2022-01-01 00:00:00'))

## Plot 1: Scatter + Line Plot 

We'll first build a line plot that showcases the changing numerical details (employment percentage, mean monthly income, etc.) across the different years. The plot should be able to filter according to school and degree.

This will allow the user to filter and compare the job acquisition performance of the same degree across different schools.

In [ ]:
df1 = df.copy()
# Prepare the data for the plot
df1 = df1[
  (df1['university'] == uni) &
  (df1['year'] >= year[0]) &
  (df1['year'] <= year[1]) &
  (df1['degree'].isin(degree)) &
  (df1['advanced'] == 0) # automatically filter out advanced further studies
]
df1['year'] = df1['year'].dt.strftime("%Y")
df1 = df1.sort_values('degree')

# Pivot the dataframe to match hvplot format
df2 = df1.pivot(index='year', columns='degree', values='basic_monthly_mean').reset_index()
cols_for_line = list(df2.columns.drop('year'))


# Plot the line graph
scatter_plot = df1.hvplot.scatter(
  x='year',
  y='basic_monthly_mean',
  by='degree',
  size=70,
  hover_cols=['year', 'basic_monthly_mean'],
  title=f'Mean Monthly Salary for Jobs in {uni}',
  xlabel='Year',
  ylabel='Mean Monthly Salary',
  color=COLOURS[:len(degree)],
  legend='top'
)

line_plot = df2.hvplot.line(
  x='year',
  y=cols_for_line,
  by='degree',
  color=COLOURS[:len(degree)]
)

# Combine the scatter plot and line plot together
plot1 = scatter_plot * line_plot
plot1

:Overlay
   .NdOverlay.I  :NdOverlay   [degree]
      :Scatter   [year]   (basic_monthly_mean)
   .NdOverlay.II :NdOverlay   [degree]
      :Curve   [year]   (value)

## Plot 2: Error Bars Plot

The error bars plot shows the 25th, median and 75th percentile for each degree. It should allow a filter on the university and degree.

This allows the user to have a better understanding of the distribution for each degree.

In [264]:
df1 = df.copy()
# Prepare the data for the plot
df1 = df1[
  (df1['university'] == uni) &
  (df1['year'] == year[1]) &
  (df1['degree'].isin(degree)) &
  (df1['advanced'] == 0) # automatically filter out advanced further studies
]
# Get the difference between the 25th/ 75th percentile and the median
df1['lower_error'] = df1['gross_monthly_median'] - df1['gross_mthly_25_percentile']
df1['upper_error'] = df1['gross_mthly_75_percentile'] - df1['gross_monthly_median']

median = df1.hvplot.scatter(
  x='degree',
  y='gross_monthly_median',
  color='#39b9e8',
  size=100,
  marker='s',
  ylabel='Gross Monthly Salary',
  xlabel='Degree',
  title=f'Errors for Gross Monthly Salary in {year[1]}',
  legend=False
)

errors = df1.hvplot.errorbars(
  x='degree',
  y='gross_monthly_median',  # Median as central value
  yerr1='lower_error',
  yerr2="upper_error",
  color='black',
  line_width=2,
  legend=False
)

errors * median

:Overlay
   .ErrorBars.I :ErrorBars   [degree]   (gross_monthly_median,lower_error,upper_error)
   .Scatter.I   :Scatter   [degree]   (gross_monthly_median)

In [257]:
df1.head()

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile,advanced
937,2021-01-01,National University of Singapore,Faculty of Arts & Social Sciences,Art,91.4,70.0,3405.0,3200.0,3600.0,3200.0,2850.0,4100.0,0
956,2021-01-01,National University of Singapore,NUS Business School,Business Administration,94.7,81.6,5199.0,3550.0,5567.0,3700.0,3125.0,7625.0,0
958,2021-01-01,National University of Singapore,NUS Business School,Accountancy,100.0,100.0,3380.0,3300.0,3515.0,3400.0,3013.0,3750.0,0
973,2021-01-01,National University of Singapore,Multi-Disciplinary Programmes,Computer Engineering,97.4,91.2,5345.0,5000.0,5473.0,5400.0,4500.0,6200.0,0


In [259]:
df10

,Category,Q1,Q2,Q3,Lower Error,Upper Error
0,Cat1,3400,3600,3800,200,200
1,Cat2,4400,4600,4800,200,200


In [258]:
# Data for error bars
data = {
    "Category": ["Cat1", "Cat2"],
    "Q1": [3400, 4400],  # 25th percentile
    "Q2": [3600, 4600],  # 50th percentile (Median)
    "Q3": [3800, 4800],  # 75th percentile
}

df10 = pd.DataFrame(data)

# Calculate error bars
df10["Lower Error"] = df10["Q2"] - df10["Q1"]
df10["Upper Error"] = df10["Q3"] - df10["Q2"]

# Plot using hvplot.errorbars
box_plot = df10.hvplot.errorbars(
    x="Category",
    y="Q2",  # Median as central value
    yerr1="Lower Error",
    yerr2="Upper Error",
    color="black",
    line_width=4,
    legend=False
) * df10.hvplot.scatter(x="Category", y="Q2", color="red", size=100, marker="s", legend=False)

box_plot.opts(title="Manual Box Plot with Error Bars", width=600, height=400)

:Overlay
   .ErrorBars.I :ErrorBars   [Category]   (Q2,Lower Error,Upper Error)
   .Scatter.I   :Scatter   [Category]   (Q2)

## Plot 3: Bar Graph

Next we'll build a bar graph to compare the employment rates between the various degrees in the __latest year__ selected. The visualisation should filter on the university and degree.

This will allow users to compare the various employment rates between each degrees without having to dive deeply and individually sort values.

In [ ]:
df1 = df.copy()
# Prepare the data for the plot
df1 = df1[
  (df1['university'] == uni) &
  (df1['year'] == year[1]) &
  (df1['degree'].isin(degree)) &
  (df1['advanced'] == 0) # automatically filter out advanced further studies
]
df1['year'] = df1['year'].dt.strftime("%Y")
df1 = df1.sort_values('employment_rate_overall')

# Plot the bar graph
plot3 = df1.hvplot.barh(
  y='employment_rate_overall',
  x='degree',
  color='#39b9e8',
  line_color=None,
  ylabel='Full-time Employment Percentage (%)',
  xlabel='Degree',
  title=f'Full-time Employment Percentage in \n{uni} ({year[1]})'
)

plot3

:Bars   [degree]   (employment_rate_overall)

## Plot 4: Dumbbell Plot

The dumbbell plot is used to emphasie gaps in values between 2 groups of data. In this case, we use it to showcase the difference between degrees that are honours degrees or cum laude achievement. Here we observe the mean salary difference.

This allows users to directly compare those degrees that have different tiers.

In [248]:
df1 = df.copy()
# Prepare the data for the plot
degree = ['Business Administration', 'Accountancy', 'Art', 'Computer Engineering']
df1 = df1[
  (df1['university'] == uni) &
  (df1['year'] == year[1]) &
  (df1['degree'].isin(degree))
]
df1['year'] = df1['year'].dt.strftime("%Y")

# Filter out degrees that have both advanced degrees
adv_filter = df1.groupby('degree').count().reset_index()
adv_filter = list(adv_filter[adv_filter['advanced'] == 2]['degree'])
df1 = df1[df1['degree'].isin(adv_filter)]

# Pivot dataframe into the advanced and not advanced columns
df1 = df1.pivot(index='degree', values='basic_monthly_mean', columns='advanced').reset_index()
df1 = df1.rename({0: 'start', 1: 'end'}, axis=1)

# Create the dumbbell plot
## Scatter plot points for advanced and not advanced values
start_points = df1.hvplot.scatter(
  x='start',
  y='degree',
  color="#39b9e8",
  size=100
)
end_points = df1.hvplot.scatter(
  x='end',
  y='degree',
  color="#91dc4c",
  size=100
)

# Create line segments connecting the points
segments = hv.Segments([
  (
    df1.loc[i, "start"],
    df1.loc[i, "degree"], 
    df1.loc[i, "end"],
    df1.loc[i, "degree"]
  ) 
for i in df1.index]).opts(color="black", line_width=2)

# Combine plots
dumbbell = (segments * start_points * end_points) \
  .opts(
    title=f"Difference in Mean Monthly Salary for Jobs \nwith Different Qualification Levels in {year[1]}",
    xlabel="Degree",
    ylabel="Mean Monthly Salary"
  )
dumbbell

:Overlay
   .Segments.I :Segments   [x0,y0,x1,y1]
   .Scatter.I  :Scatter   [start]   (degree)
   .Scatter.II :Scatter   [end]   (degree)

# Limitations

* Limited to 5 degrees due to space constraint
* Limited to 1 year in bar graph
* Boxplot is limited by the number of degrees